In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras
import csv
from keras.preprocessing.text import Tokenizer
import numpy as np
imdb = keras.datasets.imdb

Using TensorFlow backend.


In [2]:
imdb = keras.datasets.imdb
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

In [3]:
data=[]
label=[]
tweet=[]
tsvfile = open('Apple-Twitter-Sentiment-DFE.csv')
reader = csv.reader(tsvfile, delimiter=',')
for row in reader:
        #print(type(row[5]))
        r=[]
        l=[]
        if row[5]!='3' :
                for i in (row[11].split()):
                    try:
                        r.append(word_index[i])
                    except KeyError:
                        r.append(word_index["<UNK>"])
                if row[5]=='5':
                    label.append(1)
                elif row[5]=='1':
                    label.append(0)
                if row[5]!='not_relevant':
                    data.append(r)
                    tweet.append(row[11])
del data[0]
traindata=np.array(data[:1400])
valdata=np.array(data[1400:1642])
del data
trainlabel=np.array(label[:1400])
vallabel=np.array(label[1400:1642])
del label
valtweet=tweet[1400:1642]

In [4]:
traindata = keras.preprocessing.sequence.pad_sequences(traindata,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=40)

valdata = keras.preprocessing.sequence.pad_sequences(valdata,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=40)

In [5]:
vocab_size = 100000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,32,input_length=40))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(32, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 32)            3200000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 3,201,089
Trainable params: 3,201,089
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [7]:
history = model.fit(traindata,
                    trainlabel,
                    epochs=40,
                    batch_size=100,
                    verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
1400/1400 [==============================] - 2s 1ms/sample - loss: 0.6720 - acc: 0.7279
Epoch 2/40
1400/1400 [==============================] - 1s 1ms/sample - loss: 0.6318 - acc: 0.7529
Epoch 3/40
1400/1400 [==============================] - 1s 1ms/sample - loss: 0.5964 - acc: 0.7529
Epoch 4/40
1400/1400 [==============================] - 2s 1ms/sample - loss: 0.5707 - acc: 0.7529
Epoch 5/40
1400/1400 [==============================] - 2s 1ms/sample - loss: 0.5589 - acc: 0.7529
Epoch 6/40
1400/1400 [==============================] - 1s 1ms/sample - loss: 0.5535 - acc: 0.7529
Epoch 7/40
1400/1400 [==============================] - 2s 1ms/sample - loss: 0.5491 - acc: 0.7529
Epoch 8/40
1400/1400 [==============================] - 2s 1ms/sample - loss: 0.5437 - acc: 0.7529
Epoch 9/40
1400/1400 [==============================] - 2s 1ms/sample - loss: 0.5378 - acc: 0.7529
Epoch 10/40
1400/1400 [==============================] - 2s 1

In [8]:
results = model.evaluate(valdata, vallabel)

242/242 [==============================] - 0s 644us/sample - loss: 0.8015 - acc: 0.6942


In [9]:
s=model.predict(valdata)

In [10]:
print(vallabel[13])

1


In [11]:
t=[]

In [12]:
for i in s:
    if i>=0.5:
        t.append(1)
    else:
        t.append(0)

In [14]:
file =open('PredictionsSingle.csv', mode='w')
fieldnames = ['Tweet', 'Actual', 'Predicted']
writer = csv.DictWriter(file, fieldnames=fieldnames)
writer.writeheader()
for i in range(0,len(s)):
       writer.writerow({'Tweet': valtweet[i], 'Actual': vallabel[i], 'Predicted':t[i]})

In [15]:
file.close()